In [66]:
import keras
from  keras.callbacks import EarlyStopping
from copy import deepcopy
import numpy as np
from numpy import random
Alphabet="ёцыТХ)xЭ[ф0бЦЛ!УР,иМдОКИж>хълэ?"+'"'+"Щ–Ф#_ря*ЯюоЮ м-'(щсзйЗЬ/—Ш»{шаЕнгЫПСДБ<АЧ:Й];у}в…чп.«&ГНкЖеьтВ„"
def string_vectorizer(string, alphabet=Alphabet):
    vector = [[0 if char != letter else 1 for char in Alphabet] 
                  for letter in string]
    return vector
def vector_stringizer(vector, alphabet = Alphabet,return_first=True):
    answ=''
    if return_first:
        index = vector.argsort()[-1]
        answ+=Alphabet[index]
    else:
        index = vector.argsort()[-2]
        answ+=Alphabet[index]
    return answ
def batch_generator(train_set):
    global g,g0
    final_data=None
    final_label=None
    counter=0
    this_count=0
    i=0
    while True:
        data = data_onehot[train_set[i]-window_len:train_set[i],:]
        #print(data.shape)
        label = data_onehot[train_set[i],:]
        if counter==0:
            final_data=data.reshape(1,data.shape[0],data.shape[1])
            final_label=label.reshape(1,label.shape[0])
        else:
            data=data.reshape((1,data.shape[0],data.shape[1]))
            label=label.reshape((1,label.shape[0]))
            try:
                final_data=np.concatenate([final_data,data],axis=0)
            except:
                print(i)
                raise Exception(str(i)+' '+str(final_data.shape)+' '+str(data.shape))
            try:                                
                final_label=np.concatenate([final_label,label],axis=0)
            except:
                raise Exception(str(final_label.shape)+' '+str(label.shape))    

        counter+=1
        i=(i+1)%len(train_set)
        if counter==batch_size:
            if final_data.shape==(batch_size,window_len,len(Alphabet)) and final_label.shape==(batch_size,len(Alphabet)):
                yield (final_data,final_label)
                this_count+=1
                if this_count%15==0:
                    print('completed '+str(i*100/len(train_set))+' percent')
            else:
                print('ERROR')
                print(i)
                g=final_data,final_label
                raise Exception(str(g[0].shape)+' '+str(g[1].shape))
            final_data=None
            final_label=None
            counter=0
def train_model(file='Стихи+Поэмы.txt',batch_size=1500,window_len=35,n_epochs=30,model_name='secondly_trained'):
    text=open(file,'r').readlines()
    text = [i for i in text if i != '' and 'Стихотворения' not in i]
    text = [i.strip() for i in text]
    text = text[:len(text)-243]
    data=''.join(text)
    data = data.replace('\xa0',' ')

    for number in '123456789':
        data=data.replace(number,'0')
    for english_letter in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ':
        data=data.replace(english_letter, '')
    for english_letter in 'abcdefghijklmnopqrstuvwzyz':
        data=data.replace(english_letter, '')    
    b=set(data)
    c=dict()
    for symbol in b:
        c[symbol] = data.count(symbol)
    import operator
    c1 = sorted(c.items(), key=operator.itemgetter(1))
    Alphabet =''.join(c.keys())
    data_onehot = string_vectorizer(data)
    s=np.arange(window_len, len(data_onehot))
    random.seed(333)
    train_set = np.random.choice(np.arange(window_len,len(data_onehot)-window_len), int(0.85*(len(data_onehot)-window_len)), replace=False)
    test_set=np.array(list(set(np.arange(window_len,len(data_onehot)-window_len))-set(train_set)))
    callback = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='min')
    train_set = train_set[:len(train_set)-(len(train_set)%batch_size)]
    test_set = test_set[:len(test_set)-(len(test_set)%batch_size)]
    model2 = Sequential()
    model2.add(LSTM(120, input_shape=(window_len, len(Alphabet)), return_sequences=True))
    model2.add(Flatten())
    model2.add(Dropout(0.1))
    model2.add(Dense(len(Alphabet),activation='softmax'))
    model2.compile(loss='binary_crossentropy', optimizer='adam')
    model2.summary()
    model2.fit_generator(
        batch_generator(train_set), epochs=n_epochs,
        steps_per_epoch = len(train_set)//batch_size,
        validation_data = batch_generator(test_set),
        validation_steps = len(test_set)//batch_size,
        verbose=2, callbacks=[callback])
    return model2,Alphabet

In [72]:
def models_test(use_random=True,use_large=True,text_len=2000,initial_letters=None,load_model=True):
    if load_model:
        Alphabet="ёцыТХ)xЭ[ф0бЦЛ!УР,иМдОКИж>хълэ?"+'"'+"Щ–Ф#_ря*ЯюоЮ м-'(щсзйЗЬ/—Ш»{шаЕнгЫПСДБ<АЧ:Й];у}в…чп.«&ГНкЖеьтВ„"
        model=keras.models.load_model('Pushkin3.h5')
    else:
        if use_large:
            model,Alphabet=train_model('Стихи+Поэмы.txt')
        else:
            model,Alphabet=train_model('Стихи.txt')
    window_len=model.layers[0].input_shape[1]
    text=' '*window_len
    if initial_letters!=None:
        text=initial_letters[:window_len].ljust(window_len,' ')
    text1=deepcopy(text)
    text2=string_vectorizer(text1,Alphabet)
    text3=np.array(text2)
    for i in range(text_len):
        text_array = np.array(string_vectorizer(text1,Alphabet)).reshape(1,window_len,len(Alphabet))
        predicted_vector = model.predict(text_array)
        j = random.randint(0,3*window_len)
        if j%(2*window_len)!=0 or not use_random:
            predicted_char = vector_stringizer(predicted_vector[0],Alphabet, return_first=True)
        else:
            predicted_char = vector_stringizer(predicted_vector[0],Alphabet, return_first=False)
        text = text+predicted_char
        text1 = text1[1:]+predicted_char
    #if i%20==0:
        #print('Predicted '+str(i)+' letters ')
    return text

In [73]:
models_test(initial_letters='Президент России Владимир Путин призвал ',use_random=True)

'Президент России Владимир Путин приводил сон, и сладость отрадали.Но все город и волненьем страсти своей сонный,И страсть и постель и покой..Когде же страстью полей,Он старая страстью славой,В тебе сказал в нем сердце полетели,Не старик в поле не волненье.Но всё по волном полести молчаливый,От своей сердце полести,Не проститель полести молодой,Не славу пред нем и волненье,И в стратем сердце полесть,И в сердце пред нем и волненье,Как славы веселье приводит,Как славы веселье просторой,И славости не смел и поэт.В тебе страшно в темной простором,Как постишь мой долго стороны,Как под молодой городит,И страсть и не под отом страстью старины,И старик в серебе полести,На поле картины поледной,И в поле беседный своей своей,Под молите пред ней стороны,В сердце пред нем стороны страстный просторой,И в сердце пред нему старина,Что с ней старин сердце полетели,И с ней старик в сереце приводит.Он полне страшно в тебя страстью старины,И полны простором страсной стороны,И полном и старине, поэти стра